c# CRF
参考文章：
[1]https://mp.weixin.qq.com/s?__biz=MzU2Njg4MTMzNA==&mid=2247484120&idx=1&sn=695a39b128eb55d526b7e6df5dc02b94&chksm=fca4ff81cbd37697ddb68712859c4d408609c2325b0fefc814dcf5ba367b3ae148329a40c7df&token=1605957211&lang=zh_CN#rd
[2]https://pytorch-crf.readthedocs.io/en/stable/
[3]https://www.zhihu.com/question/35866596/answer/236886066
[4]https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html#sphx-glr-beginner-nlp-advanced-tutorial-py

## 前置内容：
（待补充）

## 为什么叫条件随机场？
回答这个问题需要先来看看什么是马尔可夫随机场(也叫马尔可夫网络)，如果一个位置的赋值只和与它相邻的位置的值有关，与和它不相邻的位置的值无关，那么这个随机场就是一个马尔可夫随机场。

设X与Y是随机变量，P(Y|X)是给定X时Y的条件概率分布，若随机变量Y构成的是一个马尔科夫随机场，则称条件概率分布P(Y|X)是条件随机场。在实际的应用中，比如上面的两个例子，我们一般都要求X和Y有相同的结构，如下：
X=(X1,X2,...Xn),Y=(Y1,Y2,...Yn)

比如词性标注，我们要求输出的词性序列和输入的句子中的每个词是一一对应的。
X和Y有相同的结构的CRF就构成了线性链条件随机场(Linear chain Conditional Random Fields,简称 linear-CRF)。

## CRF如何提取特征？

CRF中有两类特征函数，分别是状态特征和转移特征，状态特征用当前节点(某个输出位置可能的状态中的某个状态称为一个节点)的状态分数表示，转移特征用上一个节点到当前节点的转移分数表示。其损失函数定义如下：
CRF1.PNG

CRF损失函数的计算，需要用到真实路径分数(包括状态分数和转移分数)，其他所有可能的路径的分数(包括状态分数和转移分数)。这里的路径用词性来举例就是一句话对应的词性序列，真实路径表示真实的词性序列，其他可能的路径表示其他的词性序列。

## 总结

①
②

# 实战
## ①CRF

In [1]:
import torch
from torchcrf import CRF


C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
num_tags =  7  # 词特征长度
model = CRF(num_tags)

In [3]:
seq_length = 2  # 序列长度
batch_size = 2  # batch大小
# emissions 可以理解为状态矩阵
emissions = torch.randn(seq_length, batch_size, num_tags) #[L,B,N] 注意这个API的格式不是[B,L,N]
tags = torch.tensor([
  [0, 1], [2, 1]
], dtype=torch.long)  # (seq_length, batch_size)
model(emissions, tags)

tensor(-11.2607, grad_fn=<SumBackward0>)

In [4]:
emissions

tensor([[[-0.8303,  0.8152, -0.7888,  0.2462,  0.6199, -1.4562,  0.5228],
         [-1.4330, -0.8797, -1.3763,  1.3269, -2.2921,  1.7829,  0.7800]],

        [[ 0.3138, -0.2517, -0.0088,  1.4125,  0.9306, -2.2296, -0.3985],
         [ 0.1559,  0.1564,  0.2527,  1.3239,  0.1325, -1.2549,  2.1608]]])

In [5]:
# 初学者可能不熟悉tensor，这里调换一下位置方便理解
emissions.permute(1,0,2)[0]
# 你可以理解为CRF为中的状态矩阵

tensor([[-0.8303,  0.8152, -0.7888,  0.2462,  0.6199, -1.4562,  0.5228],
        [ 0.3138, -0.2517, -0.0088,  1.4125,  0.9306, -2.2296, -0.3985]])

In [6]:
model.decode(emissions)

[[1, 3], [5, 6]]

In [7]:
model.transitions # 模型的转移特征矩阵，代表着i->j的转移概率

Parameter containing:
tensor([[-0.0516, -0.0030,  0.0508, -0.0514,  0.0851,  0.0046, -0.0426],
        [ 0.0941,  0.0184,  0.0988, -0.0508,  0.0226, -0.0611,  0.0406],
        [ 0.0545,  0.0546, -0.0633,  0.0297, -0.0072,  0.0248, -0.0310],
        [-0.0374, -0.0802,  0.0814, -0.0032, -0.0106,  0.0453, -0.0532],
        [ 0.0060,  0.0133, -0.0704,  0.0333,  0.0032, -0.0551, -0.0200],
        [-0.0331,  0.0427,  0.0175,  0.0302, -0.0094, -0.0516,  0.0263],
        [ 0.0423, -0.0943, -0.0680,  0.0530, -0.0126, -0.0172, -0.0520]],
       requires_grad=True)

In [8]:
fin = open('../data/uni/China/mooc/train.txt', 'r', encoding='utf-8', newline='\n', errors='ignore')
lines = fin.readlines()
fin.close()

In [9]:
lines

['虽 O -999\n',
 '然 O -999\n',
 '内 B-ASP Positive\n',
 '容 I-ASP Positive\n',
 '偏 O -999\n',
 '向 O -999\n',
 '于 O -999\n',
 '初 O -999\n',
 '高 O -999\n',
 '中 O -999\n',
 '的 O -999\n',
 '生 O -999\n',
 '物 O -999\n',
 '知 O -999\n',
 '识 O -999\n',
 '但 O -999\n',
 '还 O -999\n',
 '是 O -999\n',
 '学 O -999\n',
 '到 O -999\n',
 '了 O -999\n',
 '精 O -999\n',
 '彩 O -999\n',
 '纷 O -999\n',
 '呈 O -999\n',
 '的 O -999\n',
 '东 O -999\n',
 '西 O -999\n',
 '\n',
 '非 O -999\n',
 '常 O -999\n',
 '感 O -999\n',
 '谢 O -999\n',
 '中 O -999\n',
 '国 O -999\n',
 '大 O -999\n',
 '学 O -999\n',
 '慕 O -999\n',
 '课 O -999\n',
 '给 O -999\n',
 '我 O -999\n',
 '提 O -999\n',
 '供 O -999\n',
 '再 O -999\n',
 '次 O -999\n',
 '学 O -999\n',
 '习 O -999\n',
 '的 O -999\n',
 '机 O -999\n',
 '会 O -999\n',
 '老 O -999\n',
 '师 O -999\n',
 '的 O -999\n',
 '课 O -999\n',
 '讲 B-ASP Positive\n',
 '的 O -999\n',
 '很 O -999\n',
 '细 O -999\n',
 '致 O -999\n',
 '很 O -999\n',
 '生 O -999\n',
 '动 O -999\n',
 '很 O -999\n',
 '喜 O -999\n',
 '欢 O -999\n',
 '但 O -99

In [10]:
for n,i in enumerate(lines):
    if i=="\n":
        print(1)
        print(lines[n-1])

1
西 O -999

1
谢 O -999

1
谢 O -999

1
呢 O -999

1
解 O -999

1
解 O -999

1
高 O -999

1
高 O -999

1
高 O -999

1
堂 O -999

1
堂 O -999

1
堂 O -999

1
现 O -999

1
种 O -999

1
种 O -999

1
种 O -999

1
种 O -999

1
棒 O -999

1
的 O -999

1
能 O -999

1
能 O -999

1
能 O -999

1
展 O -999

1
展 O -999

1
啊 O -999

1
懂 O -999

1
了 O -999

1
解 O -999

1
解 O -999

1
的 O -999

1
了 O -999

1
了 O -999

1
惜 O -999

1
惜 O -999

1
解 I-ASP Negative

1
解 I-ASP -999

1
点 O -999

1
点 O -999

1
谅 O -999

1
棒 O -999

1
棒 O -999

1
棒 O -999

1
人 O -999

1
欢 O -999

1
欢 O -999

1
出 O -999

1
趣 O -999

1
趣 O -999

1
多 O -999

1
多 O -999

1
多 O -999

1
多 O -999

1
看 O -999

1
看 O -999

1
好 O -999

1
好 O -999

1
习 O -999

1
棒 O -999

1
吧 O -999

1
动 O -999

1
识 I-ASP Positive

1
些 O -999

1
有 O -999

1
的 O -999

1
的 O -999

1
看 O -999

1
法 O -999

1
错 O -999

1
解 I-ASP -999

1
解 I-ASP Positive

1
懂 O -999

1
趣 O -999

1
mooc O -999

1
清 O -999

1
清 O -999

1
清 O -999

1
够 O -999

1
闷 I-ASP Negative

1
赞 O -999

1
音 O -99

In [18]:
item_token=[]
item_label=[]
cotent=[]
for n,item in enumerate(lines):
    if item=='\n':
        data = {
            'token_list':item_token,
            'label_list':item_label,
            'length':len(item_label)
        }
        item_token=[]
        item_label=[]
        cotent.append(data)
        continue
    else:
        item_token.append(item[0])
        item_label.append(item[2])

In [19]:
cotent

[{'token_list': ['虽',
   '然',
   '内',
   '容',
   '偏',
   '向',
   '于',
   '初',
   '高',
   '中',
   '的',
   '生',
   '物',
   '知',
   '识',
   '但',
   '还',
   '是',
   '学',
   '到',
   '了',
   '精',
   '彩',
   '纷',
   '呈',
   '的',
   '东',
   '西'],
  'label_list': ['O',
   'O',
   'B',
   'I',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O'],
  'length': 28},
 {'token_list': ['非',
   '常',
   '感',
   '谢',
   '中',
   '国',
   '大',
   '学',
   '慕',
   '课',
   '给',
   '我',
   '提',
   '供',
   '再',
   '次',
   '学',
   '习',
   '的',
   '机',
   '会',
   '老',
   '师',
   '的',
   '课',
   '讲',
   '的',
   '很',
   '细',
   '致',
   '很',
   '生',
   '动',
   '很',
   '喜',
   '欢',
   '但',
   '是',
   '能',
   '不',
   '能',
   '把',
   '一',
   '课',
   '时',
   '改',
   '成',
   '四',
   '十',
   '五',
   '分',
   '钟',
   '或',
   '总',
   '半',
   '个',
   '小',
   '时',
   '只',
   '有',
   '四',
   

In [20]:
#from transformers import BertTokenizer, BertModel

#tokenizer =  BertTokenizer.from_pretrained('bert-base-chinese')

In [2]:
from transformers import AutoTokenizer, BertModel
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3",cache_dir='./')


C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\lenovo\.conda\envs\CLMLF\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


KeyboardInterrupt: 

In [23]:
tokenizer(cotent[0]['token_list'], truncation=True, is_split_into_words=True, max_length=64)

{'input_ids': [101, 6006, 4197, 1079, 2159, 974, 1403, 754, 1159, 7770, 704, 4638, 4495, 4289, 4761, 6399, 852, 6820, 3221, 2110, 1168, 749, 5125, 2506, 5290, 1439, 4638, 691, 6205, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

NameError: name 'AutoTokenizer' is not defined

## ②BERT+CRF
下面我们进行一个模型的尝试

In [30]:
from torch.utils.data import Dataset
import torch.nn.utils.rnn as run_utils
from transformers import AutoTokenizer, BertModel
from transformers import BertTokenizer, BertModel
#tokenizer = BertTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")

# 这里我们使用中文RoBERT 进行embedding
# 数据文件选择中文数据集的
# 预处理方法详情说明请参考对应目录的proceess.py文件或其目录下对应的说明文件
class CommonDataset(Dataset):
    def __init__(self,data_path='../data/uni/China/mooc/',process_type=None,file_type='.txt'):
        self.tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

        self.data_path = data_path
        if process_type=='train':
            self.data_path=self.data_path+'train'+file_type
        elif process_type =='dev':
            self.data_path=self.data_path+'dev'+file_type
        elif process_type =='test':
            self.data_path=self.data_path+'test'+file_type
        else:
            assert('Wrong process_type,please make it as \'train\',\'dev\',or\'test\'')
        #self.content={}
        if file_type=='.txt':
            fin = open('../data/uni/China/mooc/train.txt', 'r', encoding='utf-8', newline='\n', errors='ignore')
            lines = fin.readlines()
            fin.close()
            item_token=[]
            item_label=[]
            self.token_list_to_id=[]
            self.list_attention=[]
            self.text_list=[]
            self.label_list=[]
            dic={'B':0,'I':1,'O':2}
            for n,item in enumerate(lines):
                if item=='\n':
                    #CLS_id = self.tokenizer.convert_tokens_to_ids(["[CLS]"])
                    #SEP_id = self.tokenizer.convert_tokens_to_ids(["[SEP]"])
                    max_len = 64 #bert截取的最大长度
                    token_result=tokenizer(cotent[0]['token_list'], truncation=True, is_split_into_words=True, max_length=max_len)
                    text_to_id=token_result['input_ids']
                    text_att=token_result['attention_mask']
                    #test_att=
                    self.text_list.append(item_token)
                    self.label_list.append(item_label)
                    self.token_list_to_id.append(text_to_id)
                    self.list_attention.append(text_att)
                    item_token=[]
                    item_label=[]
                    continue
                else:
                    item_token.append(item[0])
                    item_label.append(dic[item[2]])

    def __len__(self):
        return len(self.content)

    def __getitem__(self, index):
        token=self.token_list_to_id[index]
        text=self.text_list[index]
        att=self.list_attention[index]
        label=self.label_list[index]

        return text,token,att,label

class Collate():
    def __init__(self, opt):
        self.text_length_dynamic = 3
        if self.text_length_dynamic == 1:
            # 使用动态的长度
            self.min_length = 1
        elif self.text_length_dynamic == 0:
            # 使用固定动的文本长度
            self.min_length = opt.word_length
        pass
    def __call__(self, batch_data):
        return self._collate(batch_data)

    def _collate(self, batch_data):
        text_to_id = [torch.LongTensor(b[1]) for b in batch_data]


        label = torch.LongTensor([b[3] for b in batch_data])

        data_length = [text.size(0) for text in text_to_id]


        max_length = max(data_length)
        if max_length < self.min_length:
            # 这一步防止在后续的计算过程中，因为文本长度和mask长度不一致而出错
            text_to_id[0] = torch.cat((text_to_id[0], torch.LongTensor([0] * (self.min_length - text_to_id[0].size(0)))))
            max_length = self.min_length

        text_to_id = run_utils.pad_sequence(text_to_id, batch_first=True, padding_value=0)
        bert_attention_mask = []
        for length in data_length:
            text_mask_cell = [1] * length
            text_mask_cell.extend([0] * (max_length - length))
            bert_attention_mask.append(text_mask_cell[:])

        return text_to_id, torch.LongTensor(bert_attention_mask), label



In [31]:


from torch.utils.data import DataLoader

dataset_t = CommonDataset(process_type='train')

opt=''
opt.acc_batch_size=4
opt.cuda=True
data_type=1
data_loader_t = DataLoader(dataset_t, batch_size=opt.acc_batch_size,
                         shuffle=True if data_type == 1 else False,
                         num_workers=1, collate_fn=Collate(opt), pin_memory=True if opt.cuda else False)


KeyError: 'o'

## ③BERT+BILSTM+CRF

# ④二维CRF （grid-CRF）